# Experiment tracking

**Important:** `SQliteTracker` got a big upgrade in version `0.8.2` so ensure you are running such version or a higher one.

`SQLiteTracker` provides a powerful and flexible way to track computational (e.g., Machine Learning) experiments using a SQLite database. Allows you to use SQL as the query language, giving you a powerful tool for experiment comparison, and comes with plotting features to compare plots side-by-side and to combine plots for better comparison.

Read more about the motivations in our [blog post](https://ploomber.io/blog/experiment-tracking/), check out the [HN discussion.](https://news.ycombinator.com/item?id=33624018)

This tutorial will walk you through the features with a Machine Learning use case; however, the tracker is generic enough to be used in any other domains.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, RocCurveDisplay

In [2]:
# delete our example database, if any
db = Path('my_experiments.db')

if db.exists():
    db.unlink()

In [1]:
from sklearn_evaluation import SQLiteTracker

tracker = SQLiteTracker('my_experiments.db')

In [4]:
X, y = datasets.make_classification(200, 10, n_informative=5, class_sep=0.65)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

models = [RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()]

## Training and logging models

In [5]:
for m in models:
    model = type(m).__name__
    print(f'Fitting {model}')


    experiment = tracker.new_experiment()
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # log a dictionary with log_dict
    experiment.log_dict({'accuracy': acc, 'model': model, **m.get_params()})

Fitting RandomForestClassifier
Fitting LogisticRegression
Fitting DecisionTreeClassifier


Or use `.log(key, value)` to log individual values:

In [6]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc = accuracy_score(y_test, y_pred)

experiment = tracker.new_experiment()

# log individual values
experiment.log('accuracy', acc)
experiment.log('model', type(svc).__name__)

_ = experiment.log_dict(svc.get_params())

## Displaying latest experiments

Display the `tracker` object to show last experiments:

In [7]:
tracker

uuid,created,parameters,comment
55e0d154,2022-11-24 20:18:11,"{""accuracy"": 0.7575757575757576, ""model"": ""RandomForestClassifier"", ""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": ""sqrt"", ""max_leaf_nodes"": null, ""max_samples"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""n_estimators"": 100, ""n_jobs"": null, ""oob_score"": false, ""random_state"": null, ""verbose"": 0, ""warm_start"": false}",
71f5fbd9,2022-11-24 20:18:11,"{""accuracy"": 0.7878787878787878, ""model"": ""LogisticRegression"", ""C"": 1.0, ""class_weight"": null, ""dual"": false, ""fit_intercept"": true, ""intercept_scaling"": 1, ""l1_ratio"": null, ""max_iter"": 100, ""multi_class"": ""auto"", ""n_jobs"": null, ""penalty"": ""l2"", ""random_state"": null, ""solver"": ""lbfgs"", ""tol"": 0.0001, ""verbose"": 0, ""warm_start"": false}",
94a096ea,2022-11-24 20:18:11,"{""accuracy"": 0.7575757575757576, ""model"": ""DecisionTreeClassifier"", ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": null, ""max_leaf_nodes"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""random_state"": null, ""splitter"": ""best""}",
e0773345,2022-11-24 20:18:11,"{""accuracy"": 0.803030303030303, ""model"": ""SVC"", ""C"": 1.0, ""break_ties"": false, ""cache_size"": 200, ""class_weight"": null, ""coef0"": 0.0, ""decision_function_shape"": ""ovr"", ""degree"": 3, ""gamma"": ""scale"", ""kernel"": ""rbf"", ""max_iter"": -1, ""probability"": false, ""random_state"": null, ""shrinking"": true, ""tol"": 0.001, ""verbose"": false}",


## Querying experiments with SQL using `.query()`

You can use SQL to query your experiments. To see what's been logged, use `get_parameters_keys()`:

In [6]:
keys = tracker.get_parameters_keys()
# show first 5 keys
keys[:5]

['C', 'accuracy', 'bootstrap', 'break_ties', 'cache_size']

To generate a sample query, use `.get_sample_query()`:

In [7]:
print(tracker.get_sample_query())

SELECT
    uuid,
    json_extract(parameters, '$.C') as C,
    json_extract(parameters, '$.accuracy') as accuracy,
    json_extract(parameters, '$.bootstrap') as bootstrap,
    json_extract(parameters, '$.break_ties') as break_ties,
    json_extract(parameters, '$.cache_size') as cache_size,
    json_extract(parameters, '$.ccp_alpha') as ccp_alpha,
    json_extract(parameters, '$.class_weight') as class_weight,
    json_extract(parameters, '$.classification_report') as classification_report,
    json_extract(parameters, '$.coef0') as coef0,
    json_extract(parameters, '$.confusion_matrix') as confusion_matrix,
    json_extract(parameters, '$.criterion') as criterion,
    json_extract(parameters, '$.decision_function_shape') as decision_function_shape,
    json_extract(parameters, '$.degree') as degree,
    json_extract(parameters, '$.dual') as dual,
    json_extract(parameters, '$.fit_intercept') as fit_intercept,
    json_extract(parameters, '$.gamma') as gamma,
    json_extract(para

To execute a query, use `.query()`:

In [8]:
ordered = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
""")
ordered

,model,accuracy
uuid,,
e0773345,SVC,0.803030
71f5fbd9,LogisticRegression,0.787879
55e0d154,RandomForestClassifier,0.757576
94a096ea,DecisionTreeClassifier,0.757576


The query method returns a data frame with "uuid" as the index:

In [9]:
type(ordered)

pandas.core.frame.DataFrame

## Storing plots

You can log a confusion matrix and classification reports:

In [10]:
%%capture

def fit(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    experiment = tracker.new_experiment()
    experiment.log_dict({'accuracy': acc, 'model': type(model).__name__, **model.get_params()})
    
    # log plots     
    experiment.log_confusion_matrix(y_test, y_pred)
    experiment.log_classification_report(y_test, y_pred)
    
    # log generic matplotlib figure
    roc = RocCurveDisplay.from_estimator(model, X_test, y_test)
    experiment.log_figure('roc', roc.figure_)
    
fit(model=RandomForestClassifier(n_estimators=100))
fit(model=RandomForestClassifier(n_estimators=10))

In [11]:
tracker.recent(2)

,created,parameters,comment
uuid,,,
66069e95,2022-11-24 20:18:12,"{""accuracy"": 0.8181818181818182, ""model"": ""Ran...",None
38ebfea2,2022-11-24 20:18:12,"{""accuracy"": 0.7272727272727273, ""model"": ""Ran...",None


## Rendering plots in table view

The `.query()` method also allows rendering plots in the table view:

In [12]:
results = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy,
       json_extract(parameters, '$.confusion_matrix') AS cm,
       json_extract(parameters, '$.roc') AS roc
FROM experiments
ORDER BY created DESC
LIMIT 2
""", as_frame=False, render_plots=True)

results

uuid,model,accuracy,cm,roc
66069e95,RandomForestClassifier,0.818182,,
38ebfea2,RandomForestClassifier,0.727273,,


## Side-by-side comparison

From the `.query()` results, you can extract a given column for a side by side comparison:

In [13]:
results.get("cm")

You can change the labels in the tabs with the `index_by` argument:

In [14]:
results.get("cm", index_by="accuracy")

## Combining plots

With a side-by-side comparison, it might be hard to spot the model performance differents, you can get individual experiments, extract their plots and combine them:

In [15]:
# get the uuids for the latest 2 experiments
uuid1, uuid2 = tracker.recent(2).index

# get the experiments
one = tracker.get(uuid1)
another = tracker.get(uuid2)

Combine statistics from both confusion matrices:

In [16]:
one["confusion_matrix"] + another["confusion_matrix"]

Get confusion matrix differences:

In [17]:
one["confusion_matrix"] - another["confusion_matrix"]

Combine classification reports:

In [18]:
one["classification_report"] + another["classification_report"]

You can also retrieve generic figures (logged with `log_figure`):

In [19]:
one["roc"]

However, note that plot combination (`plot1 + plot2` and `plot1 - plot2`) is only supported by plots logged via the `experiment.log_*` and not by the generic `experiment.log_figure` method.

## Adding comments

In [20]:
one.comment('This is some comment')

In [21]:
tracker.query("""
SELECT uuid,
       comment,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
WHERE comment is not NULL
""")

,comment,model,accuracy
uuid,,,
66069e95,This is some comment,RandomForestClassifier,0.818182


## Pandas integration

### Getting recent experiments

The recent method also returns a data frame:

In [22]:
df = tracker.recent()
df

,created,parameters,comment
uuid,,,
66069e95,2022-11-24 20:18:12,"{""accuracy"": 0.8181818181818182, ""model"": ""Ran...",This is some comment
38ebfea2,2022-11-24 20:18:12,"{""accuracy"": 0.7272727272727273, ""model"": ""Ran...",None
55e0d154,2022-11-24 20:18:11,"{""accuracy"": 0.7575757575757576, ""model"": ""Ran...",None
71f5fbd9,2022-11-24 20:18:11,"{""accuracy"": 0.7878787878787878, ""model"": ""Log...",None
94a096ea,2022-11-24 20:18:11,"{""accuracy"": 0.7575757575757576, ""model"": ""Dec...",None


Pass `normalize=True` to convert the nested JSON dictionary into columns:

In [23]:
df = tracker.recent(normalize=True)
df

,created,accuracy,model,bootstrap,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,...,fit_intercept,intercept_scaling,l1_ratio,max_iter,multi_class,penalty,solver,tol,splitter,comment
uuid,,,,,,,,,,,,,,,,,,,,,
66069e95,2022-11-24 20:18:12,0.818182,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This is some comment
38ebfea2,2022-11-24 20:18:12,0.727273,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
55e0d154,2022-11-24 20:18:11,0.757576,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
71f5fbd9,2022-11-24 20:18:11,0.787879,LogisticRegression,NaN,NaN,None,NaN,NaN,NaN,NaN,...,True,1.0,NaN,100.0,auto,l2,lbfgs,0.0001,NaN,None
94a096ea,2022-11-24 20:18:11,0.757576,DecisionTreeClassifier,NaN,0.0,None,gini,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,best,None


### Querying

You can also use the `.query()` method with `as_frame=True` (default value) to get a `pandas.DataFrame`

In [24]:
df = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
LIMIT 3
""")
df

,model,accuracy
uuid,,
66069e95,RandomForestClassifier,0.818182
e0773345,SVC,0.803030
71f5fbd9,LogisticRegression,0.787879
